<a href="https://colab.research.google.com/github/Alenkar14/ComCie/blob/main/Proyecto_Bayesiana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Paso 1: Instalar solo dependencias esenciales
!pip install -q numpy pandas matplotlib seaborn

# Paso 2: Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración estética
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

# =================================================
# 1. SIMULACIÓN DE DATOS MEJORADA
# =================================================
print("\nGenerando datos de simulación...")
np.random.seed(123)
n_asegurados = 10000

# Grupos de edad con distribución más realista
dist_edades = {
    '30-40': 0.35,
    '41-50': 0.30,
    '51-60': 0.20,
    '61-70': 0.15
}

edades = np.concatenate([
    np.random.randint(30, 40, int(n_asegurados * dist_edades['30-40'])),
    np.random.randint(41, 50, int(n_asegurados * dist_edades['41-50'])),
    np.random.randint(51, 60, int(n_asegurados * dist_edades['51-60'])),
    np.random.randint(61, 70, int(n_asegurados * dist_edades['61-70']))
])

genero = np.random.choice(['Hombre', 'Mujer'], size=n_asegurados, p=[0.55, 0.45])
diabetes = np.random.choice([0, 1], size=n_asegurados, p=[0.7, 0.3])
obesidad = np.random.choice([0, 1], size=n_asegurados, p=[0.6, 0.4])
pandemia = np.concatenate([np.zeros(8000), np.ones(2000)])

# Mortalidad simulada con efecto edad y comorbilidades
lambda_basal = 0.005 + (edades - 30) * 0.0001
comorb_factor = 1 + 0.5 * diabetes + 0.3 * obesidad
lambda_total = np.where(
    pandemia == 1,
    lambda_basal * comorb_factor * 1.8,
    lambda_basal * comorb_factor
)
muertes = np.random.poisson(lambda_total)

# Crear DataFrame
data = pd.DataFrame({
    'edad': edades,
    'grupo_edad': pd.cut(edades, [30, 40, 50, 60, 70], right=False),
    'genero': genero,
    'diabetes': diabetes,
    'obesidad': obesidad,
    'pandemia': pandemia,
    'muertes': muertes
})

print("Resumen de datos simulados:")
print(data.describe())

# =================================================
# 2. SIMULACIONES Y ANÁLISIS
# =================================================
print("\nRealizando simulaciones de Monte Carlo...")

# 2.1. Simulación Monte Carlo para primas
n_simulaciones = 10000
siniestros = []
prima_95 = []

for i in range(n_simulaciones):
    # Simular nueva cohorte
    edad_sim = np.random.choice(edades, size=1000)
    diabetes_sim = np.random.choice([0, 1], size=1000, p=[0.7, 0.3])
    obesidad_sim = np.random.choice([0, 1], size=1000, p=[0.6, 0.4])
    pandemia_sim = np.random.choice([0, 1], size=1000, p=[0.8, 0.2])

    # Calcular mortalidad
    lambda_basal_sim = 0.005 + (edad_sim - 30) * 0.0001
    comorb_factor_sim = 1 + 0.5 * diabetes_sim + 0.3 * obesidad_sim
    lambda_total_sim = np.where(
        pandemia_sim == 1,
        lambda_basal_sim * comorb_factor_sim * 1.8,
        lambda_basal_sim * comorb_factor_sim
    )

    # Simular muertes
    muertes_sim = np.random.poisson(lambda_total_sim)
    siniestros.append(muertes_sim.sum())
    prima_95.append(np.percentile(muertes_sim, 95))

prima_global = np.percentile(siniestros, 95)
prima_media = np.mean(prima_95)
print(f"\nPrima global estimada (P95): {prima_global:.0f}")
print(f"Prima media por cohorte (P95): {prima_media:.0f}")

# =================================================
# 3. GRÁFICOS COMPLETOS
# =================================================
print("\nGenerando gráficos para el informe...")

# 3.1. Distribución de siniestros
plt.figure(figsize=(10, 6))
plt.hist(siniestros, bins=30, alpha=0.7, color='skyblue', density=True)
plt.axvline(prima_global, color='red', linestyle='--', linewidth=2,
            label=f'Prima P95: {prima_global:.0f}')

# Añadir distribución teórica
try:
    from scipy.stats import gamma
    shape, loc, scale = gamma.fit(siniestros, floc=0)
    x = np.linspace(min(siniestros), max(siniestros), 1000)
    pdf = gamma.pdf(x, shape, loc, scale)
    plt.plot(x, pdf, 'k-', linewidth=2, label='Distribución Gamma')
except:
    pass

plt.title('Distribución de Siniestralidad Total', fontsize=16)
plt.xlabel('Siniestralidad total', fontsize=12)
plt.ylabel('Densidad de probabilidad', fontsize=12)
plt.legend()
plt.tight_layout()
plt.savefig('distribucion_siniestros.png', dpi=150, bbox_inches='tight')
plt.close()

# 3.2. Simulaciones de brotes epidémicos
def brote_sir(R0, dias=180, poblacion=1000):
    S = poblacion - 1
    I = 1
    R = 0
    historico = [I]

    for _ in range(dias):
        nuevos_infectados = np.random.binomial(S, min(0.5, R0 * I / poblacion))
        nuevos_recuperados = np.random.binomial(I, 0.1)

        S = max(0, S - nuevos_infectados)
        I = I + nuevos_infectados - nuevos_recuperados
        R = R + nuevos_recuperados

        historico.append(I)
        if I <= 0: break

    return historico

plt.figure(figsize=(10, 6))
for _ in range(50):
    R0 = np.random.lognormal(mean=np.log(1.5), sigma=0.3)
    brote = brote_sir(R0)
    plt.plot(brote, alpha=0.4, color='steelblue')

plt.title('Simulaciones de Propagación Epidémica', fontsize=16)
plt.xlabel('Días', fontsize=12)
plt.ylabel('Número de infectados', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('propagacion_epidemica.png', dpi=150, bbox_inches='tight')
plt.close()

# 3.3. Comparación de primas por escenario
escenarios = {
    'Normal': {'pandemia': 0.0, 'comorb': 0.0},
    'Pandemia': {'pandemia': 1.0, 'comorb': 0.0},
    'Comorbilidad': {'pandemia': 0.0, 'comorb': 1.0},
    'Completo': {'pandemia': 1.0, 'comorb': 1.0}
}

primas = {}
for nombre, params in escenarios.items():
    siniestros_esc = []
    for _ in range(1000):
        # Simular población
        edad_sim = np.random.choice(edades, size=100)
        comorb_sim = np.random.choice([0, 1], size=100, p=[0.5, 0.5]) if params['comorb'] else np.zeros(100)

        # Calcular mortalidad
        lambda_basal_sim = 0.005 + (edad_sim - 30) * 0.0001
        efecto_comorb = 0.5 * comorb_sim  # 50% aumento por comorbilidad
        efecto_pandemia = np.full(100, params['pandemia']) * 0.8  # 80% aumento por pandemia

        lambda_total_sim = lambda_basal_sim * (1 + efecto_comorb + efecto_pandemia)

        # Simular muertes
        muertes_sim = np.random.poisson(lambda_total_sim)
        siniestros_esc.append(muertes_sim.sum())

    primas[nombre] = np.percentile(siniestros_esc, 95)

# Gráfico de comparación
plt.figure(figsize=(10, 6))
plt.bar(primas.keys(), primas.values(),
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
        alpha=0.8)

# Añadir etiquetas de valor
for i, v in enumerate(primas.values()):
    plt.text(i, v + 5, f'{v:.0f}', ha='center', fontsize=12)

plt.title('Comparación de Primas por Escenario', fontsize=16)
plt.ylabel('Prima estimada', fontsize=12)
plt.ylim(0, max(primas.values()) + 50)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('comparacion_primas.png', dpi=150, bbox_inches='tight')
plt.close()

# 3.4. Impacto de comorbilidades por edad
impacto_edad = data.groupby(['grupo_edad', 'diabetes'])['muertes'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='grupo_edad', y='muertes', hue='diabetes', data=impacto_edad,
            palette=['#1f77b4', '#ff7f0e'], alpha=0.8)

plt.title('Impacto de Diabetes en Mortalidad por Grupo de Edad', fontsize=16)
plt.xlabel('Grupo de Edad', fontsize=12)
plt.ylabel('Mortalidad Promedio', fontsize=12)
plt.legend(title='Diabetes', loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('impacto_edad.png', dpi=150, bbox_inches='tight')
plt.close()

# 3.5. Distribución de edades
plt.figure(figsize=(10, 6))
sns.histplot(edades, bins=20, kde=True, color='teal')
plt.title('Distribución de Edades en la Población Asegurada', fontsize=16)
plt.xlabel('Edad', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('distribucion_edades.png', dpi=150, bbox_inches='tight')
plt.close()

# 3.6. Efecto pandemia por grupo de edad
pandemia_edad = data.groupby(['grupo_edad', 'pandemia'])['muertes'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='grupo_edad', y='muertes', hue='pandemia', data=pandemia_edad,
            palette=['#1f77b4', '#ff7f0e'], alpha=0.8)

plt.title('Impacto de Pandemia en Mortalidad por Grupo de Edad', fontsize=16)
plt.xlabel('Grupo de Edad', fontsize=12)
plt.ylabel('Mortalidad Promedio', fontsize=12)
plt.legend(title='Pandemia', loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('pandemia_edad.png', dpi=150, bbox_inches='tight')
plt.close()

print("\nTodas las gráficas generadas con éxito!")
print("Gráficas disponibles:")
!ls *.png


Generando datos de simulación...
Resumen de datos simulados:
               edad     diabetes      obesidad     pandemia       muertes
count  10000.000000  10000.00000  10000.000000  10000.00000  10000.000000
mean      46.291400      0.29770      0.400500      0.20000      0.009600
std       11.113714      0.45727      0.490024      0.40002      0.099543
min       30.000000      0.00000      0.000000      0.00000      0.000000
25%       37.000000      0.00000      0.000000      0.00000      0.000000
50%       45.000000      0.00000      0.000000      0.00000      0.000000
75%       55.000000      1.00000      1.000000      0.00000      0.000000
max       69.000000      1.00000      1.000000      1.00000      2.000000

Realizando simulaciones de Monte Carlo...

Prima global estimada (P95): 15
Prima media por cohorte (P95): 0

Generando gráficos para el informe...


/tmp/ipython-input-24-4121349940.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  impacto_edad = data.groupby(['grupo_edad', 'diabetes'])['muertes'].mean().reset_index()
/tmp/ipython-input-24-4121349940.py:241: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pandemia_edad = data.groupby(['grupo_edad', 'pandemia'])['muertes'].mean().reset_index()



Todas las gráficas generadas con éxito!
Gráficas disponibles:
comparacion_primas.png	 distribucion_siniestros.png  pandemia_edad.png
distribucion_edades.png  impacto_edad.png	      propagacion_epidemica.png
